In [ ]:
# import beberapa library
import sys
import random
import copy
import numpy as np

In [ ]:
# Representasi class state
class State:
  # Inisialisasi variabel
  def __init__(self, route:[], distance:int=0):
    self.route = route
    self.distance = distance
  # Membandingkan state
  def __eq__(self, other):
    for i in range(len(self.route)):
      if(self.route[i] != other.route[i]):
        return False
    return True
  # Sorting state
  def __lt__(self, other):
    return self.distance < other.distance
  # Cetak state
  def __repr__(self):
    return ('({0}, {1})\n'.format(self.route, self.distance))
  # Membuat shallow copy
  def copy(self):
    return State(self.route, self.distance)
  # Membuat deep copy
  def deepcopy(self):
    return State(copy.deepcopy(self.route), copy.deepcopy(self.distance))
  # Update distance
  def update_distance(self, matrix, home):
    # Reset distance
    self.distance = 0
    # Keep track of departing city
    from_index = home
    # Loop all cities in the current route
    for i in range(len(self.route)):
      self.distance += matrix[from_index][self.route[i]]
      from_index = self.route[i]
    # Add the distance back to home
    self.distance += matrix[from_index][home]
# This class represent a city (used when we need to delete cities)

In [ ]:
class City:
  # Inisialisasi city
  def __init__(self, index:int, distance:int):
    self.index = index
    self.distance = distance
  # Storing city
  def __lt__(self, other):
    return self.distance < other.distance

In [ ]:
# Return true with probability p
def probability(p):
  return p > random.uniform(0.0, 1.0)

# Schedule function for simulated annealing
def exp_schedule(k=20, lam=0.005, limit=1000):
  return lambda t: (k * np.exp(-lam * t) if t < limit else 0)

# Get the best random solution from a population
def get_random_solution(matrix:[], home:int, city_indexes:[], size:int, use_weights:bool=False):
  # Create a list with city indexes
  cities = city_indexes.copy()
  # Remove the home city
  cities.pop(home)
  # Create a population
  population = []
  for i in range(size):
    if(use_weights == True):
      state = get_random_solution_with_weights(matrix, home)
    else:
      # Shuffle cities at random
      random.shuffle(cities)
      # Create a state
      state = State(cities[:])
      state.update_distance(matrix, home)
    # Add an individual to the population
    population.append(state)
  # Sort population
  population.sort()
  # Return the best soluation
  return population[0]

# Get best solution by distance
def get_best_solution_by_distance(matrix:[], home:int):
  # Variables
  route = []
  from_index = home
  length = len(matrix) - 1
  # Loop until route is complete
  while len(route) < length:
    # Get a matrix row
    row = matrix[from_index]
    # Create a list with cities
    cities = {}
    for i in range(len(row)):
      cities[i] = City(i, row[i])
    # Remove cities that already is assigned to the route
    del cities[home]
    for i in route:
      del cities[i]
    # Sort cities
    sorted = list(cities.values())
    sorted.sort()
    # Add the city with the shortest distance
    from_index = sorted[0].index
    route.append(from_index)
  # Create a new state and update the dsitance
  state = State(route)
  state.update_distance(matrix, home)
  # Return a state
  return state

# Get a random solution by using weights
def get_random_solution_with_weights(matrix:[], home:int):
  # Variables
  route = []
  from_index = home
  length = len(matrix) - 1
  # Loop until route is complete
  while len(route) < length:
    # Get a matrix row
    row = matrix[from_index]
    # Create a list with cities
    cities = {}
    for i in range(len(row)):
      cities[i] = City(i, row[i])
    # Remove cities that already is assigned to the route
    del cities[home]
    for i in route:
      del cities[i]
    # Get the total weight
    total_weight = 0
    for key, city in cities.items():
      total_weight += city.distance
    # Add weights
    weights = []
    for key, city in cities.items():
      weights.append(total_weight / city.distance)
    # Add a city at random
    from_index = random.choices(list(cities.keys()), weights=weights)[0]
    route.append(from_index)
  # Create a new state and update the distance
  state = State(route)
  state.update_distance(matrix, home)
  # Return a state
  return state

# Mutate a solution
def mutate(matrix:[], home:int, state:State, mutation_rate:float=0.01):
  # Create a copy of the state
  mutated_state = state.deepcopy()
  # Loop all the states in a route
  for i in range(len(mutated_state.route)):
    # Check if we should do a mutation
    if(random.random() < mutation_rate):
      # Swap two cities
      j = int(random.random() * len(state.route))
      city_1 = mutated_state.route[i]
      city_2 = mutated_state.route[j]
      mutated_state.route[i] = city_2
      mutated_state.route[j] = city_1
    # Update the distance
    mutated_state.update_distance(matrix, home)
    # Return a mutated state
    return mutated_state

# Simulated annealing
def simulated_annealing(matrix:[], home:int, initial_state:State, mutation_rate:float=0.01, schedule=exp_schedule()):
  # Keep track of the best state
  best_state = initial_state
  # Loop a large number of times (int.max)
  for t in range(sys.maxsize):
    # Get a temperature is 0
    T = schedule(t)
    if T == 0:
      return best_state
    # Mutate the best state
    neighbor = mutate(matrix, home, best_state, mutation_rate)
    # Calculate the change in e
    delta_e = best_state.distance - neighbor.distance
    # Check if we should update the best state
    if delta_e > 0 or probability(np.exp(delta_e / T)):
      best_state = neighbor

In [ ]:
def main():
  # daftar kota
  cities = ['jakarta', 'bandung', 'semarang', 'jogjakarta', 'surabaya', 'malang',]
  city_indexes = [0,1,2,3,4,5]
  # index mulai
  home = 2
  # jarak antar kota yang di masukkan dalam list
  matrix = [[0, 94, 289, 348, 495, 536],
            [94, 0, 192, 346, 478, 486],
            [289, 192, 0, 82, 193, 243],
            [348, 346, 82, 0, 213, 206],
            [495, 476, 193, 213, 0, 62],
            [536, 486, 243, 206, 62, 0]]

  # Mencari rute terbaik dari jarak
  state = get_best_solution_by_distance(matrix, home)
  print('-- Best solution by distance --')
  print(cities[home], end='')
  for i in range(0, len(state.route)):
    print(' -> ' + cities[state.route[i]], end='')
  print(' -> ' + cities[home], end='')
  print('\n\nTotal distance : {0} miles'.format(state.distance))
  print()

  # Mencari rute terbaik dari niali random
  state = get_random_solution(matrix, home, city_indexes, 100)
  print('-- Best random solution --')
  print(cities[home], end='')
  for i in range(0, len(state.route)):
    print(' -> ' + cities[state.route[i]], end='')
  print(' -> ' + cities[home], end='')
  print('\n\nTotal distance : {0} miles'.format(state.distance))
  print()

  # Mencari rute terbaik dengan menggunakan weight
  state = get_random_solution(matrix, home, city_indexes, 100, use_weights=True)
  print('-- Best random solution with weights')
  print(cities[home], end='')
  for i in range(0, len(state.route)):
    print(' -> ' + cities[state.route[i]], end='')
  print(' -> ' + cities[home], end='')
  print('\n\nTotal distance : {0} miles'.format(state.distance))
  print()

  # Mencari solusi terbaik menggunakan simulated annelling
  state = get_best_solution_by_distance(matrix, home)
  state = simulated_annealing(matrix, home, state, 0.1)
  print('-- Simulated annealing solution')
  print(cities[home], end='')
  for i in range(0, len(state.route)):
    print(' -> ' + cities[state.route[i]], end='')
  print(' -> ' + cities[home], end='')
  print('\n\nTotal distance : {0} miles'.format(state.distance))
  print()

if __name__ == "__main__":
  main()

-- Best solution by distance --
semarang -> jogjakarta -> malang -> surabaya -> bandung -> jakarta -> semarang

Total distance : 1209 miles

-- Best random solution --
semarang -> bandung -> jakarta -> jogjakarta -> malang -> surabaya -> semarang

Total distance : 1095 miles

-- Best random solution with weights
semarang -> surabaya -> malang -> jogjakarta -> jakarta -> bandung -> semarang

Total distance : 1095 miles

-- Simulated annealing solution
semarang -> surabaya -> malang -> jogjakarta -> bandung -> jakarta -> semarang

Total distance : 1190 miles

